## Parametrizing synapse models
* https://nest-simulator.readthedocs.io/en/latest/tutorials/pynest_tutorial/part_3_connecting_networks_with_synapses.html#pynest-tutorial-3

In [1]:
import nest
import matplotlib.pyplot as plt
%matplotlib inline

[admin:1424045] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.admin.1000/jf.0/1374552064/shared_mem_cuda_pool.admin could be created.
[admin:1424045] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.8.0-post0.dev0
 Built: Oct  2 2024 11:54:50

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
nest.SetDefaults("stdp_synapse",{"tau_plus": 15.0})

In [3]:
nest.CopyModel("stdp_synapse","layer1_stdp_synapse",{"Wmax": 90.0}) # create a new model with a higher Wmax

In [4]:
nest.GetDefaults("stdp_synapse") # check the default values

{'alpha': 1.0,
 'delay': 1.0,
 'element_type': 'synapse',
 'has_delay': True,
 'Kplus': 0.0,
 'lambda': 0.01,
 'mu_minus': 1.0,
 'mu_plus': 1.0,
 'num_connections': 0,
 'receptor_type': 0,
 'requires_symmetric': False,
 'sizeof': 96,
 'synapse_model': 'stdp_synapse',
 'synapse_modelid': 47,
 'tau_plus': 15.0,
 'Wmax': 100.0,
 'weight': 1.0,
 'weight_recorder': NodeCollection(<empty>)}

In [5]:
nest.GetDefaults("iaf_psc_alpha") # check the default values

{'archiver_length': 0,
 'available': (0,),
 'beta_Ca': 0.001,
 'C_m': 250.0,
 'Ca': 0.0,
 'capacity': (0,),
 'E_L': -70.0,
 'element_type': 'neuron',
 'elementsize': 688,
 'frozen': False,
 'global_id': 0,
 'I_e': 0.0,
 'instantiations': (0,),
 'local': True,
 'model': 'iaf_psc_alpha',
 'model_id': 50,
 'node_uses_wfr': False,
 'post_trace': 0.0,
 'recordables': ('I_syn_ex', 'I_syn_in', 'V_m'),
 'synaptic_elements': {},
 't_ref': 2.0,
 't_spike': -1.0,
 'tau_Ca': 10000.0,
 'tau_m': 10.0,
 'tau_minus': 20.0,
 'tau_minus_triplet': 110.0,
 'tau_syn_ex': 2.0,
 'tau_syn_in': 2.0,
 'thread': -1,
 'thread_local_id': -1,
 'type_id': 'iaf_psc_alpha',
 'V_m': -70.0,
 'V_min': -inf,
 'V_reset': -70.0,
 'V_th': -55.0,
 'vp': -1}

In [6]:
epop1 = nest.Create("iaf_psc_alpha", 10, params={"tau_minus": 30.0})
epop2 = nest.Create("iaf_psc_alpha", 10, params={"tau_minus": 30.0})

In [7]:
K = 20
conn_dict = {"rule": "fixed_indegree", "indegree": K}
syn_dict = {"synapse_model": "stdp_synapse", "alpha": 1.0}
nest.Connect(epop1, epop2, conn_dict, syn_dict)

### Distributing synapse parameters
* parameters associated with the distribution can be set (for example mean). Here we show an example where the parameters alpha and weight of the stdp synapse are uniformly distributed.
* See further details in the NEST documentation: https://nest-simulator.readthedocs.io/en/latest/neurons/parametrization.html#param-ex

In [8]:
alpha_min = 0.1
alpha_max = 2.
w_min = 0.5
w_max = 5.

syn_dict = {"synapse_model": "stdp_synapse",
            "alpha": nest.random.uniform(min=alpha_min, max=alpha_max),
            "weight": nest.random.uniform(min=w_min, max=w_max),
            "delay": 1.0}
nest.Connect(epop1, epop2, "all_to_all", syn_dict)

### Querying the synapses
* The function GetConnections(source=None, target=None, synapse_model=None) returns a SynapseCollection representing connection identifiers that match the given specifications. 

In [9]:
nest.GetConnections(epop1)

In [10]:
nest.GetConnections(target=epop2)

In [11]:
nest.GetConnections(epop1, epop2, "stdp_synapse")

In [12]:
conns = nest.GetConnections(epop1, synapse_model="stdp_synapse")
print(conns)

 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1       12    stdp_synapse    1.000   1.000
      1       18    stdp_synapse    1.000   1.000
      1       11    stdp_synapse    1.000   1.000
      1       12    stdp_synapse    4.976   1.000
      1       11    stdp_synapse    1.000   1.000
      1       17    stdp_synapse    1.000   1.000
      1       20    stdp_synapse    3.147   1.000
      1       11    stdp_synapse   0.6025   1.000
      1       15    stdp_synapse    3.532   1.000
      1       12    stdp_synapse    1.000   1.000
      1       16    stdp_synapse   0.6878   1.000
      1       12    stdp_synapse    1.000   1.000
      1       13    stdp_synapse    4.212   1.000
      1       16    stdp_synapse    1.000   1.000
      1       11    stdp_synapse    1.000   1.000
     ⋮        ⋮               ⋮        ⋮       ⋮ 
     10       15    stdp_synapse    1.000   1.000
     10       14    stdp_synapse    1.533   1.00

In [13]:
print(conns.get().keys()) # actually, conns is a dictionary and contain much more information than just showed above

dict_keys(['Kplus', 'Wmax', 'alpha', 'delay', 'lambda', 'mu_minus', 'mu_plus', 'port', 'receptor', 'sizeof', 'source', 'synapse_id', 'synapse_model', 'target', 'target_thread', 'tau_plus', 'weight'])


In [14]:
targets = conns.get("target")
print(targets) # returns the list of targets

[12, 18, 11, 12, 11, 17, 20, 11, 15, 12, 16, 12, 13, 16, 11, 20, 20, 16, 13, 16, 19, 18, 16, 14, 18, 19, 16, 14, 16, 14, 17, 19, 15, 18, 19, 17, 18, 14, 14, 16, 14, 19, 14, 14, 13, 18, 16, 19, 13, 16, 19, 13, 20, 15, 17, 12, 20, 18, 11, 12, 18, 20, 14, 14, 13, 14, 16, 13, 20, 18, 12, 19, 16, 14, 19, 19, 17, 17, 17, 17, 17, 17, 14, 12, 19, 17, 17, 15, 19, 18, 13, 11, 18, 16, 15, 17, 11, 20, 14, 20, 15, 17, 12, 11, 15, 14, 19, 12, 20, 12, 16, 17, 18, 19, 14, 15, 13, 18, 15, 19, 20, 18, 11, 17, 17, 17, 18, 16, 12, 16, 12, 12, 15, 16, 14, 15, 12, 15, 17, 20, 15, 14, 14, 18, 20, 13, 13, 19, 13, 13, 18, 13, 19, 19, 13, 11, 13, 14, 12, 20, 12, 17, 12, 15, 16, 19, 14, 16, 11, 11, 11, 16, 11, 11, 17, 15, 20, 11, 20, 11, 12, 14, 14, 20, 12, 20, 13, 15, 11, 18, 12, 18, 17, 15, 12, 13, 16, 12, 13, 13, 16, 13, 19, 13, 18, 18, 11, 14, 13, 17, 11, 20, 11, 20, 20, 20, 12, 11, 11, 20, 12, 19, 12, 13, 13, 19, 13, 18, 14, 14, 15, 19, 19, 19, 15, 17, 14, 16, 16, 15, 17, 18, 18, 17, 15, 17, 16, 16, 16, 16,

In [15]:
conn_vals = conns.get(["target","weight"])
print(conn_vals) # returns a dictionary with the keys "target" and "weight"

{'target': [12, 18, 11, 12, 11, 17, 20, 11, 15, 12, 16, 12, 13, 16, 11, 20, 20, 16, 13, 16, 19, 18, 16, 14, 18, 19, 16, 14, 16, 14, 17, 19, 15, 18, 19, 17, 18, 14, 14, 16, 14, 19, 14, 14, 13, 18, 16, 19, 13, 16, 19, 13, 20, 15, 17, 12, 20, 18, 11, 12, 18, 20, 14, 14, 13, 14, 16, 13, 20, 18, 12, 19, 16, 14, 19, 19, 17, 17, 17, 17, 17, 17, 14, 12, 19, 17, 17, 15, 19, 18, 13, 11, 18, 16, 15, 17, 11, 20, 14, 20, 15, 17, 12, 11, 15, 14, 19, 12, 20, 12, 16, 17, 18, 19, 14, 15, 13, 18, 15, 19, 20, 18, 11, 17, 17, 17, 18, 16, 12, 16, 12, 12, 15, 16, 14, 15, 12, 15, 17, 20, 15, 14, 14, 18, 20, 13, 13, 19, 13, 13, 18, 13, 19, 19, 13, 11, 13, 14, 12, 20, 12, 17, 12, 15, 16, 19, 14, 16, 11, 11, 11, 16, 11, 11, 17, 15, 20, 11, 20, 11, 12, 14, 14, 20, 12, 20, 13, 15, 11, 18, 12, 18, 17, 15, 12, 13, 16, 12, 13, 13, 16, 13, 19, 13, 18, 18, 11, 14, 13, 17, 11, 20, 11, 20, 20, 20, 12, 11, 11, 20, 12, 19, 12, 13, 13, 19, 13, 18, 14, 14, 15, 19, 19, 19, 15, 17, 14, 16, 16, 15, 17, 18, 18, 17, 15, 17, 16, 

### Tips for numbers and variables
* If you have a parameters section in your script, and group the variable names according to function (e.g. neuronal parameters, synaptic parameters, stimulation parameters,…) then it is much easier to find and check them. Similarly, if you need to share parameters between simulation scripts, it is much less error-prone to define all the variable names in a separate parameters file, which the individual scripts can import. Thus a good rule of thumb is that numbers should only be visible in distinct parameter files or parameter sections, otherwise they should be represented by variables.

* It's good to have parameters.py as a separate file, and import it in the main script. This way, you can easily change the parameters without having to search through the entire script.

### Loop over neurons
* don't use neuron is like below
```python
for n in range(1,len(neuronpop)+1):
    nest.SetStatus(NodeCollection([n]), {"V_m": -67.0})
```
* instead use
```python
nest.SetStatus(neuronpop, {"V_m": -67.0}) # or, even better
neuronpop.set(V_m=-67.0)
```
* If you want to loop over neurons, you can use the following code:
```python
for n in neuronpop:
    my_weird_function(n)
```